In [3]:
from pymatgen.analysis.wulff import WulffShape

# Import the necessary tools to generate surfaces
from pymatgen.core.surface import Lattice, SlabGenerator, Structure, generate_all_slabs

from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.io.cif import CifWriter

from ase.visualize import view

from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.io.ase import AseAtomsAdaptor

In [4]:
lattice = Lattice.cubic(3.508)
Ni = Structure(
    lattice,
    ["Ni", "Ni", "Ni", "Ni"],
    [[0, 0, 0], [0, 0.5, 0], [0.5, 0, 0], [0, 0, 0.5]],
)
structure = Ni

In [3]:
slabgen = SlabGenerator(Ni, (1, 1, 1), 10, 10)

all_slabs = slabgen.get_slabs()

In [7]:
surface_energies_Ni = surface_energies_Ni = {
    (3, 2, 0): 2.3869,
    (1, 1, 0): 2.2862,
    (3, 1, 0): 2.3964,
    (2, 1, 0): 2.3969,
    (3, 3, 2): 2.0944,
    (1, 0, 0): 2.2084,
    (2, 1, 1): 2.2353,
    (3, 2, 2): 2.1242,
    (3, 2, 1): 2.3183,
    (2, 2, 1): 2.1732,
    (3, 3, 1): 2.2288,
    (3, 1, 1): 2.3039,
    (1, 1, 1): 1.9235,
}
'''surface_energies_Ni = surface_energies_Ni = {
    (1, 0, 0): 2.2084,
}'''
miller_list = surface_energies_Ni.keys()
e_surf_list = surface_energies_Ni.values()

# We can now construct a Wulff shape with an accuracy up to a max Miller index of 3
wulffshape = WulffShape(Ni.lattice, miller_list, e_surf_list)

# Let's get some useful information from our wulffshape object
print(
    "shape factor: %.3f, anisotropy: \
%.3f, weighted surface energy: %.3f J/m^2"
    % (
        wulffshape.shape_factor,
        wulffshape.anisotropy,
        wulffshape.weighted_surface_energy,
    )
)


# If we want to see what our Wulff shape looks like
#wulffshape.show()

shape factor: 5.178, anisotropy: 0.070, weighted surface energy: 2.035 J/m^2


## Make sized cluster

In [10]:
import numpy as np

lattice =  np.array(  [[0.000000000000E+00 ,  0.210850000000E+01 ,  0.210850000000E+01],
   [0.210850000000E+01 ,  0.000000000000E+00  , 0.210850000000E+01],
   [0.210850000000E+01 ,  0.210850000000E+01 ,  0.000000000000E+00]])
structure = Structure(lattice,['Mg','O'],[[0.,0.,0.],[0.5,0.5,0.5]])
structure = SpacegroupAnalyzer(structure).get_conventional_standard_structure()

wulffshape = WulffShape(Ni.lattice, miller_list, e_surf_list)
atom_density = structure.num_sites/structure.volume
wulffshape.volume
#wulffshape.wulff_convex.points
wulff_atoms_0 = wulffshape.volume * atom_density
wulff_atoms_1 = 100

new_volume = 100/atom_density
volume_ratio = new_volume/wulffshape.volume
ratio = volume_ratio**1/3

shape = ConvexHull((wulffshape.wulff_convex.points/(wulffshape.volume**(1/3)))*(new_volume**(1/3)))
shape.volume*atom_density

100.00000000000007

# Make wulff-shaped NP

In [9]:
from scipy.spatial import Delaunay, ConvexHull
import numpy as np
poly = np.array([[0,0.5,0],[1.8,1,1],[2,2,2.1],[3,3.4,3]])
poly= np.array(wulffshape.get_line_in_facet(wulffshape.facets[1]))
test = []
for i in range(1000):
    point = np.array([i,i,i])
    test.append(Delaunay(wulffshape.wulff_convex.points).find_simplex(point) >= 0)

In [105]:
lattice =  np.array(  [[0.000000000000E+00 ,  0.210850000000E+01 ,  0.210850000000E+01],
   [0.210850000000E+01 ,  0.000000000000E+00  , 0.210850000000E+01],
   [0.210850000000E+01 ,  0.210850000000E+01 ,  0.000000000000E+00]])
structure = Structure(lattice,['Mg','O'],[[0.,0.,0.],[0.5,0.5,0.5]])
structure = SpacegroupAnalyzer(structure).get_conventional_standard_structure()

supercell_matrix = np.identity(3)*10
structure.make_supercell(supercell_matrix)
center_of_mass = np.average(structure.cart_coords,axis=0)
structure.translate_sites(np.arange(structure.num_sites),-center_of_mass,frac_coords=False,to_unit_cell= False)
structure.num_sites

8000

In [106]:
shape = Delaunay(wulffshape.wulff_convex.points*(new_volume**(1/3)/wulffshape.volume**(1/3)))
#shape = Delaunay(wulffshape.wulff_convex.points*(wulffshape.volume**(1/3))*(new_volume**(1/3))
sites_to_remove = []
for i,coord in enumerate(structure.cart_coords):
    if (shape.find_simplex(coord) >= 0) == False:
        sites_to_remove.append(i)
structure.remove_sites(sites_to_remove)
structure.num_sites

80

In [108]:
view(AseAtomsAdaptor().get_atoms(structure))

<Popen: returncode: None args: ['/Users/brunocamino/miniconda3/envs/test_env...>

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [119]:
from pymatgen.analysis.local_env import BrunnerNN_real, CrystalNN

In [125]:
CrystalNN(structure).get_nn_shell_info(structure,0,2)

[{'site': PeriodicSite: Mg (-5.2713, -1.0543, 1.0542) [-0.1250, -0.0250, 0.0250],
  'image': (0, 0, 0),
  'weight': 1.0000291802820345,
  'site_index': 30},
 {'site': PeriodicSite: Mg (-3.1628, -1.0543, 3.1627) [-0.0750, -0.0250, 0.0750],
  'image': (0, 0, 0),
  'weight': 1.000072950508118,
  'site_index': 11},
 {'site': PeriodicSite: Mg (-3.1628, -1.0543, -1.0543) [-0.0750, -0.0250, -0.0250],
  'image': (0, 0, 0),
  'weight': 1.148628452586877,
  'site_index': 10},
 {'site': PeriodicSite: Mg (-3.1628, 1.0542, 1.0542) [-0.0750, 0.0250, 0.0250],
  'image': (0, 0, 0),
  'weight': 1.0,
  'site_index': 2},
 {'site': PeriodicSite: Mg (-1.0543, -1.0543, 1.0542) [-0.0250, -0.0250, 0.0250],
  'image': (0, 0, 0),
  'weight': 2.0,
  'site_index': 33},
 {'site': PeriodicSite: Mg (1.0542, -3.1628, 1.0542) [0.0250, -0.0750, 0.0250],
  'image': (0, 0, 0),
  'weight': 1.0,
  'site_index': 4},
 {'site': PeriodicSite: Mg (-1.0543, -3.1628, 3.1627) [-0.0250, -0.0750, 0.0750],
  'image': (0, 0, 0),
  'we

In [128]:
np.unique(np.round(structure.distance_matrix,5))

array([ 0.     ,  2.1085 ,  2.98187,  3.65203,  4.217  ,  4.71475,
        5.16475,  5.96374,  6.3255 ,  6.66766,  6.9931 ,  7.30406,
        7.6023 ,  7.88928,  8.434  ,  8.69357,  8.94561,  9.19074,
        9.4295 ,  9.66236, 10.5425 , 10.75128, 10.95609])

In [15]:
from mendeleev import *

In [17]:
Mg.atomic_number

12